In [13]:
import os 
os.chdir("../")

In [8]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataTransformationConfig:
    rootDir:Path
    DataPath:Path
    TokonizerName:Path



In [2]:
from textSummarizer.constants import *
from textSummarizer.utils.common import read_yaml,create_directories

In [16]:
class configurationManager:
    def __init__(self,config_filepath=ConfigFilePath,params_filepath=paramsFilePath):
        self.config=read_yaml(config_filepath)
        self.params=read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])
    def getDataTransformationConfig(self)-> DataTransformationConfig:
        config= self.config.data_transformation

        create_directories([config.root_dir])
        data_transformationConfig= DataTransformationConfig(
            rootDir=config.root_dir,
            DataPath=config.data_path,
            TokonizerName=config.tokenizer_name
        )
        
        return data_transformationConfig


In [3]:
import os
from transformers import AutoTokenizer
from datasets import load_dataset,load_from_disk
from textSummarizer.logging import logger

In [26]:
class DataTransformation:
    def __init__(self,config=DataTransformationConfig) -> None:
        self.config=config
        self.tokenizer=AutoTokenizer.from_pretrained(config.TokonizerName)


    
    def convert_examples_to_features(self,example_batch):
        input_encodings = self.tokenizer(example_batch['dialogue'] , max_length = 1024, truncation = True )
        
        with self.tokenizer.as_target_tokenizer():
            target_encodings = self.tokenizer(example_batch['summary'], max_length = 128, truncation = True )
            
        return {
            'input_ids' : input_encodings['input_ids'],
            'attention_mask': input_encodings['attention_mask'],
            'labels': target_encodings['input_ids']
        }
    

    def convert(self):
        dataset_samsum = load_from_disk(self.config.DataPath)
        dataset_samsum_pt = dataset_samsum.map(self.convert_examples_to_features, batched = True)
        dataset_samsum_pt.save_to_disk(os.path.join(self.config.rootDir,"samsum_dataset"))
    

In [27]:
try:
    config = configurationManager()
    data_transformation_config = config.getDataTransformationConfig()
    data_transformation = DataTransformation(config=data_transformation_config)
    data_transformation.convert()
except Exception as e:
    raise e

[2024-09-20 12:30:53,639; INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-09-20 12:30:53,641; INFO: common: yaml file: params.yaml loaded successfully]
[2024-09-20 12:30:53,643; INFO: common: created directory at: artifacts]
[2024-09-20 12:30:53,645; INFO: common: created directory at: artifacts/data_transformation]


Map:   0%|          | 0/14732 [00:00<?, ? examples/s]c:\Users\PCS\Desktop\NLP end-to-end Project\endToend-NLP-Project\.venv\Lib\site-packages\transformers\tokenization_utils_base.py:4126: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(
Saving the dataset (1/1 shards): 100%|██████████| 818/818 [00:00<00:00, 58432.81 examples/s]
